In [102]:
from transformers import AutoTokenizer, XLMRobertaXLForTokenClassification
import torch
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")

# model = XLMRobertaXLForTokenClassification.from_pretrained("facebook/xlm-roberta-xl")

In [103]:
# inputs = tokenizer(
#     "HuggingFace is a company based in Paris and New York",
#     add_special_tokens=False,
#     return_tensors="pt",
# )

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_token_class_ids = logits.argmax(-1)

# # Note that tokens are classified rather then input words which means that
# # there might be more predicted token classes than words.
# # Multiple token classes might account for the same word
# predicted_tokens_classes = [
#     model.config.id2label[t.item()] for t in predicted_token_class_ids[0]
# ]

# labels = predicted_token_class_ids
# loss = model(**inputs, labels=labels).loss

In [120]:
import pandas as pd

unsplit_dataset = pd.read_json("pii_parsed_43k.json")

# dataset = pd.read_json("pii_parsed_43k.json")
dataset = unsplit_dataset.sample(frac=0.8, random_state=1)

eval_dataset = unsplit_dataset.drop(dataset.index)

dataset = dataset.reset_index(drop=True)
eval_dataset.reset_index(drop=True, inplace=True)

dataset = dataset.drop("document", axis="columns")
eval_dataset = eval_dataset.drop("document", axis="columns")

dataset.head()

,full_text,tokens,trailing_whitespace,labels
0,The franchise located on Moore Spurs 424 has b...,"[the, franchise, located, on, moore, spurs, 42...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-STREET_ADDRESS, I-STREET_ADDRES..."
1,I am requesting an overview of the intellectua...,"[i, am, requesting, an, overview, of, the, int...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"Dear coordinator, I wish to donate to Walsh, M...","[dear, coordinator, ,, i, wish, to, donate, to...","[True, False, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Literacy volunteers needed in Fort Elyssafield...,"[literacy, volunteers, needed, in, fort, elyss...","[True, True, True, True, True, False, True, Tr...","[O, O, O, O, I-STREET_ADDRESS, I-STREET_ADDRES..."
4,A new order for school cafeteria supplies has ...,"[a, new, order, for, school, cafeteria, suppli...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [121]:
dataset.columns

Index(['full_text', 'tokens', 'trailing_whitespace', 'labels'], dtype='object')

In [122]:
labels_list = dataset["labels"].explode().unique()
labels_list

array(['O', 'B-STREET_ADDRESS', 'I-STREET_ADDRESS', 'B-PHONE_NUM',
       'I-PHONE_NUM', 'B-USERNAME', 'I-USERNAME', 'I-NAME_STUDENT',
       'B-NAME_STUDENT', 'B-EMAIL', 'I-EMAIL', 'B-ID_NUM', 'I-ID_NUM',
       'B-URL_PERSONAL', 'I-URL_PERSONAL'], dtype=object)

In [123]:
example = dataset.iloc[2]
example

full_text              Dear coordinator, I wish to donate to Walsh, M...
tokens                 [dear, coordinator, ,, i, wish, to, donate, to...
trailing_whitespace    [True, False, True, True, True, True, True, Tr...
labels                 [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: 2, dtype: object

In [124]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>',
 '▁de',
 'ar',
 '▁coordina',
 'tor',
 '▁',
 ',',
 '▁i',
 '▁wish',
 '▁to',
 '▁donat',
 'e',
 '▁to',
 '▁wal',
 'sh',
 '▁',
 ',',
 '▁m',
 'cc',
 'ull',
 'ough',
 '▁and',
 '▁ste',
 'u',
 'ber',
 '▁for',
 '▁the',
 '▁gift',
 'ed',
 '▁education',
 '▁programs',
 '▁',
 '.',
 '▁attached',
 '▁is',
 '▁a',
 '▁fund',
 '▁transfer',
 '▁via',
 '▁zim',
 'bab',
 'we',
 '▁dollar',
 '▁',
 '.',
 '▁my',
 '▁i',
 'ban',
 '▁is',
 '▁se',
 '73',
 '04',
 '989',
 '500',
 '90',
 '705',
 '70',
 '310',
 '48',
 '▁',
 '.',
 '</s>']

In [125]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

labels = [i for i in example["labels"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {
    0: "O",
    1: "B-NAME_STUDENT",
    2: "I-NAME_STUDENT",
    3: "B-EMAIL",
    4: "I-EMAIL",
    5: "B-USERNAME",
    6: "I-USERNAME",
    7: "B-ID_NUM",
    8: "I-ID_NUM",
    9: "B-PHONE_NUM",
    10: "I-PHONE_NUM",
    11: "B-URL_PERSONAL",
    12: "I-URL_PERSONAL",
    13: "B-STREET_ADDRESS",
    14: "I-STREET_ADDRESS",
}
label2id = {
    "O": 0,
    "B-NAME_STUDENT": 1,
    "I-NAME_STUDENT": 2,
    "B-EMAIL": 3,
    "I-EMAIL": 4,
    "B-USERNAME": 5,
    "I-USERNAME": 6,
    "B-ID_NUM": 7,
    "I-ID_NUM": 8,
    "B-PHONE_NUM": 9,
    "I-PHONE_NUM": 10,
    "B-URL_PERSONAL": 11,
    "I-URL_PERSONAL": 12,
    "B-STREET_ADDRESS": 13,
    "I-STREET_ADDRESS": 14,
}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "facebook/xlm-roberta-xl",
    num_labels=15,
    id2label=id2label,
    label2id=label2id,
)

print(torch.backends.mps.is_available())

Some weights of XLMRobertaXLForTokenClassification were not initialized from the model checkpoint at facebook/xlm-roberta-xl and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True


In [ ]:
training_args = TrainingArguments(
    output_dir="bs_pi",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

  0%|          | 0/3808 [00:00<?, ?it/s]

KeyError: 2565